## 预备知识

### `tf.where`

`tf.where(判别条件,A,B)`是`tensorflow`中的条件语句成了返回A，否则返回B

```python
tf.where(?, A, B)
```

In [1]:
import tensorflow as tf

a = tf.constant([1,2,3,1,1])
b = tf.constant([0,3,4,1,0])
tf.where(a > b, a, b)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 3, 4, 1, 1], dtype=int32)>

### `np.random.RandomState.rand`

`np.random.RandomState.rand(维度)`可以返回一个[0,1)的随机数，若是维度为空则返回的是一个标量。

In [3]:
import numpy as np

rdm = np.random.RandomState(seed=1) #设置随机种子，保证每次生成的随机数相同
a = rdm.rand()
a

0.417022004702574

In [4]:
b = rdm.rand(2,3) #返回一个2行3列的随机数矩阵
b

array([[7.20324493e-01, 1.14374817e-04, 3.02332573e-01],
       [1.46755891e-01, 9.23385948e-02, 1.86260211e-01]])

### `np.vstack`

`np.vstack(数组1, 数组2)`可以将两个数组拼接。

In [5]:
import numpy as np

a = np.array([1,2,3])
b = np.array([3,4,5])
np.vstack((a,b))

array([[1, 2, 3],
       [3, 4, 5]])

### `np.mgrid、np.ravel、np.c_`生成网格坐标点

`np.mgrid[起始值:结束值:步长, 起始值:结束值:步长, ...]`生成指定范围的矩阵，可以同时生成多个矩阵，用`,`隔开。

`x.ravel()`将x拉伸到一维数组

`np.c_[数组1,数组2,....]`使得返回的间隔数值点配对，可以配对多个数组

In [6]:
import numpy as np

x,y = np.mgrid[1:3:1, 3:6:0.4]
print(x)
print(y)

[[1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2.]]
[[3.  3.4 3.8 4.2 4.6 5.  5.4 5.8]
 [3.  3.4 3.8 4.2 4.6 5.  5.4 5.8]]


In [7]:
x = x.ravel()
y = y.ravel()
print(x)
print(y)

[1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.]
[3.  3.4 3.8 4.2 4.6 5.  5.4 5.8 3.  3.4 3.8 4.2 4.6 5.  5.4 5.8]


In [8]:
grid = np.c_[x, y]
print(grid)

[[1.  3. ]
 [1.  3.4]
 [1.  3.8]
 [1.  4.2]
 [1.  4.6]
 [1.  5. ]
 [1.  5.4]
 [1.  5.8]
 [2.  3. ]
 [2.  3.4]
 [2.  3.8]
 [2.  4.2]
 [2.  4.6]
 [2.  5. ]
 [2.  5.4]
 [2.  5.8]]


## 激活函数

激活函数可以增加模型的表达度，因为$y = xw + b$是一个线性函数，即使有多层神经元首尾相接构成深层网络，但依旧是线性模型，模型的表达度收到限制，所以加入`非线性函数`的激活函数使得模型不再是输入`x`的线性组合，增加了模型表达度。

### 好的激活函数
- **非线性**：激活函数非线性时，多层神经网络可逼近所有函数
- **可微分**：优化器大多使用梯度下降更新参数，如果激活函数不可微那么就无法更新参数
- **单调性**：当激活函数是单调的，能保证单层网络的损失函数是凸函数，可收敛
- **近似恒等**：$f(x) \approx x$也就是激活函数的输出值应当约等于输入值，当参数初始化为随机小值时，神经网络更稳定

### 激活函数输出值的范围
- 激活函数输出为`有限值`时（分类问题），基于梯度的优化方法更稳定
- 激活函数输出为`无限值`时（回归问题），建议调小学习率

### `sigmoid`

`tf.nn.sigmoid(x)`激活函数可以将输出限制在0-1之间。

$$
f(x) = \frac{1}{1 + e^{-x}}
$$

特点是：
- 容易造成梯度消失：因为`sigmoid`函数的导数在0-0.25之间，在神经网络中更新参数时会采用逐层求导让后相乘，若网络的激活函数是`sigmoid`就会有多个0-0.25之间的数相乘，使得结果无限趋近于零，造成了梯度消失，网络参数无法更新。
- 输出非0均值，收敛慢
- 存在幂运算，计算时间长

### `tanh`

`tf.math.tanh(x)`激活函数可以将输出均值限制在0附近。
$$
f(x) = \frac{1 - e^{-2x}}{1 + e^{-2x}}
$$

特点是：
- 输出均值是0
- 容易造成梯度消失：原因和`sigmoid`一样
- 存在幂运算，计算时间长

### `relu`

`tf.nn.relu()`激活函数是一个分段函数，输入小于0时的输出都是0。
$$
f(x) = max(x,0) = \begin{cases} 0 & x<0 \\\\ x & x>=0\end{cases}
$$
优点：
- 在正区间时，解决了梯度消失
- 只需要判断输入是否大于0，计算快
- 收敛速度快`sigmoid`

缺点：
- 输出的均值不是0，收敛慢
- `Dead RelU`问题：当输入存在过多负数时，输出为0，反向传播的梯度就是0，参数无法更新，造成神经元死亡

### `Leaky Relu`

`tf.nn.leaky_relu(x)`可以解决`relu`输入为负数时，输出为0，造成神经元死亡的问题。
$$
f(x) = max(ax,x)
$$

`leaky_relu`在负区间的输出不再是0。拥有`relu`的所有优点，还解决了`Dead Relu`问题。但在实际操作中还是经常选择`relu`。

## 总结
- 首选`relu`函数
- 学习率设置比较小最好
- 输入特征标准化，即让输入数据集的均值是0，标准差是1，的正态分布
- 初始参数中心话，即让随机生成的参数满足均值是0，标准差是$\sqrt{\frac{2}{当前层输入特征数}}$的正态分布